# 配置与编译
**Configure and Build**

在本节中，我们将演示 XiangShan 的配置和编译流程。

In this section, we present the workflow for configuring and building XiangShan.

## Configure

借助 Chisel 面向对象的硬件设计语言提供的灵活性，XiangShan 可以提供相当多的可配置参数供研究者和企业定制。

Benefiting from the flexibility provided by the Chisel, an OOP HDL, XiangShan offers a wide range of configurable parameters for researchers and industry customization.

以 Kunminghu-V3 的前端设计为例，我们采用了一个层次化的参数系统，首先由设计人员定义每个模块的参数类和默认参数，如下所示：

Taking the frontend design of Kunminghu-V3 as an example, we employ a hierarchical parameter system where designers first define parameter classes and default parameters for each module, as shown below:

```scala
// xiangshan/frontend/FrontendParameters.scala
case class FrontendParameters(
    FetchBlockSize:   Int              = 64, // bytes
    bpuParameters:    BpuParameters    = BpuParameters(),
    icacheParameters: ICacheParameters = ICacheParameters(),
    // ...
) { ... }

// xiangshan/frontend/icache/Parameters.scala
case class ICacheParameters(
    NumSets:   Int     = 256,
    NumWays:   Int     = 4,
    Replacer:  String  = "setplru", // "random", "setlru", "setplru"
    EnableEcc: Boolean = true,      // whether to enable ECC or parity check
    // ...
) { ... }
```

随后，对于研究者和企业用户来说，可以在 `top/Configs.scala` 中对这些参数类进行实例化，并快速产生不同配置：

Subsequently, for researchers and industry users, these parameter classes can be instantiated in `top/Configs.scala` to quickly generate different configurations:

```scala
// 默认配置
// Default configuration
class DefaultConfig extends Config(
    // ...
    frontendParameters = FrontendParameters()
    // ...
)

// 用于快速功能验证的最小化配置，各项资源都较小，加快编译和仿真速度
// Minimal configuration for quick functional verification, with smaller resources to speed up compilation and simulation
class MinimalConfig extends Config(
    // ...
    frontendParameters = FrontendParameters(
      icacheParameters = ICacheParameters(
        NumSets   = 64,
        NumWays   = 2,
        Replacer  = "random",
        EnableEcc = false
      )
    )
    // ...
)
```

最终可以在编译时通过 `make CONFIG=XXXConfig` 来编译不同的配置。

Finally, different configurations can be compiled at build time using `make CONFIG=XXXConfig`.

## Wait a minute!

由于一些历史原因，XiangShan 的编译依赖一些环境变量，如 `NOOP_HOME`。我们提供了一个 `env.sh` 脚本来帮您配置这些环境变量。如前所述，在 Notebook 中每个单元格的环境都是独立的，故每个单元格都需要重新执行该脚本。

Due to some historical reasons, the build process of XiangShan depends on certain environment variables, such as `NOOP_HOME`. We provide an `env.sh` script to help you set up these environment variables. As mentioned earlier, each cell in the Notebook has an independent environment, so this script needs to be executed in each cell.

您可以运行下面的单元格来查看这些环境变量：

You can run the following cell to view these environment variables:

In [ ]:
%%bash
source ../env.sh

env | grep _HOME

## Build

配置好环境变量以后，我们可以直接进入 `${NOOP_HOME}` 来编译 XiangShan。

After the environment setup, go to `$NOOP_HOME` to build XiangShan.

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

# 警告⚠️：构建 XiangShan 资源消耗较大。
# 在参考配置（16 CPU 核，128 GB 内存）下，DefaultConfig 大约需要 40 分钟编译。
# 如果您正在现场参加本次 tutorial，请勿运行下面的构建命令。
# Warning⚠️：Building XiangShan is highly resource‑intensive.
# Under the reference setup (16 CPU cores, 128 GB memory), DefaultConfig takes about 40 minutes to build.
# If you are reading this notebook on-site of the tutorial, please do not run the build command below.
make emu -j16 CONFIG=MinimalConfig

# 其余 make 选项，供参考
# Additional make options
# CONFIG=MinimalConfig  XiangShan configuration
# EMU_THREADS=4         Simulation thread count
# EMU_TRACE=1           Enable waveforms
# WITH_DRAMSIM=1        Simulate DRAM with DRAMSim3
# WITH_CHISELDB=1       Enable ChiselDB
# WITH_CONSTANTIN=1     Enable Constantin

以上命令会生成 build/emu 和 build/rtl 等文件

- build/rtl/*.sv 是使用 chisel 编译出的 verilog 文件
- build/emu 是进一步使用 verilator 编译出的仿真可执行文件

The commands above will generate outputs like `build/emu` and `build/rtl`.

- build/rtl/*.sv is Verilog files generated by Chisel.
- build/emu is a simulation executable further compiled with Verilator.